## Federated Learning with one corrupted node: median
Using one balanced and one unbalanced dataset with one corrupted node (5%, 25% and 50% corrupted samples) to test different aggregation functions and determine the more robust one. 

In [2]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [4]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [5]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [6]:
def aggregate(grad_list):
    
    # Compute the median gradient for each layer
    med_grad = [
        tf.numpy_function(lambda grads: np.median(grads, axis=0), [tf.stack(layer_grads)], tf.float32)
        for layer_grads in zip(*grad_list)
    ]
    return med_grad

In [7]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3836997398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [8]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

#### 5A 5% 

In [9]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3687049449.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3687049449.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3687049449.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3687049449.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option o

In [10]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [11]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Amed.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [12]:
global_model = build_model((24,1,1))

In [13]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [15]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Amed.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 173ms/step - loss: 0.4550 - accuracy: 0.8707 - val_loss: 0.2285 - val_accuracy: 0.9369
Epoch 2/5
31/31 [==============================] - 5s 167ms/step - loss: 0.1898 - accuracy: 0.9533 - val_loss: 0.1798 - val_accuracy: 0.9592
Epoch 3/5
31/31 [==============================] - 6s 178ms/step - loss: 0.1656 - accuracy: 0.9651 - val_loss: 0.1622 - val_accuracy: 0.9633
Epoch 4/5
31/31 [==============================] - 6s 179ms/step - loss: 0.1543 - accuracy: 0.9669 - val_loss: 0.1539 - val_accuracy: 0.9653
Epoch 5/5
31/31 [==============================] - 5s 171ms/step - loss: 0.1476 - accuracy: 0.9676 - val_loss: 0.1494 - val_accuracy: 0.9657
Epoch 1/5
31/31 [==============================] - 5s 160ms/step - loss: 0.0358 - accuracy: 0.9897 - val_loss: 0.3443 - val_accuracy: 0.8025
Epoch 2/5
31/31 [==============================] - 5s 157ms/step - loss: 0.0236 - accuracy: 0.9909 - val_loss: 0.4366 - val_accuracy: 0.6613
Epoch 3/5
31/

In [16]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06691569089889526, 0.4485682249069214], [0.05725914612412453, 0.46658557653427124], [0.05533405765891075, 0.4274962842464447]]
Accuracy for iterations:  [[0.9691343307495117, 0.7738502025604248], [0.9720120429992676, 0.7521252632141113], [0.9748020768165588, 0.7949393391609192]]
F1 for iterations:  [[0.96907056257358, 0.7554715856687966], [0.9719681107465905, 0.72787197302513], [0.9747699338127015, 0.7811387610664655]]
Precision for iterations:  [[0.970221054115231, 0.8292692346163768], [0.9727156962330306, 0.8181768445573764], [0.9753060270434274, 0.841100703673347]]
Recall for iterations:  [[0.9691343597534254, 0.7738501780135145], [0.9720120365771714, 0.7521252633873429], [0.974802068421514, 0.7949393300879168]]


#### 5B 10%

In [17]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/2463029928.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/2463029928.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/2463029928.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/2463029928.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [18]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [19]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Amed.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [20]:
global_model = build_model((24,1,1))

In [21]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [22]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [23]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Amed.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 249ms/step - loss: 0.4871 - accuracy: 0.8352 - val_loss: 0.2880 - val_accuracy: 0.9100
Epoch 2/5
31/31 [==============================] - 8s 243ms/step - loss: 0.2651 - accuracy: 0.9287 - val_loss: 0.2475 - val_accuracy: 0.9396
Epoch 3/5
31/31 [==============================] - 8s 268ms/step - loss: 0.2438 - accuracy: 0.9399 - val_loss: 0.2349 - val_accuracy: 0.9418
Epoch 4/5
31/31 [==============================] - 7s 237ms/step - loss: 0.2328 - accuracy: 0.9421 - val_loss: 0.2273 - val_accuracy: 0.9424
Epoch 5/5
31/31 [==============================] - 8s 257ms/step - loss: 0.2261 - accuracy: 0.9426 - val_loss: 0.2247 - val_accuracy: 0.9425
Epoch 1/5
31/31 [==============================] - 7s 215ms/step - loss: 0.0445 - accuracy: 0.9893 - val_loss: 0.5502 - val_accuracy: 0.6659
Epoch 2/5
31/31 [==============================] - 7s 211ms/step - loss: 0.0243 - accuracy: 0.9902 - val_loss: 0.3455 - val_accuracy: 0.7673
Epoch 3/5
31/

In [24]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.05049462243914604, 0.351478636264801], [0.07023373246192932, 0.4887283146381378], [0.05930818244814873, 0.2516592741012573]]
Accuracy for iterations:  [[0.9728227257728577, 0.8347925543785095], [0.9708288311958313, 0.7519981265068054], [0.974254310131073, 0.8714669942855835]]
F1 for iterations:  [[0.9727837449695576, 0.8273961723240721], [0.9707727317440585, 0.7277151380453881], [0.9742197222524658, 0.8677772285101247]]
Precision for iterations:  [[0.9734314129700049, 0.8649459379121415], [0.9718046650406867, 0.8180750152503118], [0.9748025513826907, 0.8896848809784373]]
Recall for iterations:  [[0.9728227526366531, 0.8347925597616799], [0.9708288293552251, 0.751998110876989], [0.9742542873002424, 0.8714669766765967]]


#### 5B 25%

In [25]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3855986297.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3855986297.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3855986297.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10604/3855986297.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [26]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [27]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Amed.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [28]:
global_model = build_model((24,1,1))

In [29]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [30]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [31]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Amed.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 244ms/step - loss: 0.4551 - accuracy: 0.8538 - val_loss: 0.2205 - val_accuracy: 0.9108
Epoch 2/5
31/31 [==============================] - 7s 234ms/step - loss: 0.1498 - accuracy: 0.9636 - val_loss: 0.1231 - val_accuracy: 0.9765
Epoch 3/5
31/31 [==============================] - 7s 229ms/step - loss: 0.1164 - accuracy: 0.9774 - val_loss: 0.1103 - val_accuracy: 0.9776
Epoch 4/5
31/31 [==============================] - 7s 233ms/step - loss: 0.1052 - accuracy: 0.9789 - val_loss: 0.1033 - val_accuracy: 0.9786
Epoch 5/5
31/31 [==============================] - 7s 226ms/step - loss: 0.0996 - accuracy: 0.9796 - val_loss: 0.1001 - val_accuracy: 0.9787
Epoch 1/5
31/31 [==============================] - 7s 225ms/step - loss: 0.0319 - accuracy: 0.9899 - val_loss: 0.3505 - val_accuracy: 0.7823
Epoch 2/5
31/31 [==============================] - 7s 215ms/step - loss: 0.0229 - accuracy: 0.9909 - val_loss: 0.3052 - val_accuracy: 0.7820
Epoch 3/5
31/

In [32]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.037744056433439255, 0.30084630846977234], [0.07391563802957535, 0.6586554050445557], [0.0651039332151413, 0.5782988667488098]]
Accuracy for iterations:  [[0.9832159876823425, 0.8607134819030762], [0.9692804217338562, 0.701500415802002], [0.972172737121582, 0.7411538362503052]]
F1 for iterations:  [[0.9832096387142932, 0.8562097192925908], [0.9692161591417824, 0.658271347893325], [0.972128241436249, 0.7132048262937432]]
Precision for iterations:  [[0.9832727804796078, 0.8814826660015413], [0.9703893942707573, 0.7950497465978846], [0.9728986964229371, 0.8140293191190128]]
Recall for iterations:  [[0.9832159864442432, 0.8607135072295284], [0.9692804347190979, 0.7015003996221754], [0.972172719039411, 0.7411538182082394]]


#### 5B 5% 

In [18]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16348/3503541895.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16348/3503541895.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16348/3503541895.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')


In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Bmed.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bmed.hdf5')

Epoch 1/5
31/31 [==============================] - 11s 361ms/step - loss: 0.4603 - accuracy: 0.8307 - val_loss: 0.2359 - val_accuracy: 0.9394
Epoch 2/5
31/31 [==============================] - 10s 323ms/step - loss: 0.1960 - accuracy: 0.9515 - val_loss: 0.1720 - val_accuracy: 0.9622
Epoch 3/5
31/31 [==============================] - 8s 256ms/step - loss: 0.1646 - accuracy: 0.9651 - val_loss: 0.1575 - val_accuracy: 0.9649
Epoch 4/5
31/31 [==============================] - 10s 332ms/step - loss: 0.1522 - accuracy: 0.9673 - val_loss: 0.1519 - val_accuracy: 0.9658
Epoch 5/5
31/31 [==============================] - 10s 333ms/step - loss: 0.1461 - accuracy: 0.9675 - val_loss: 0.1463 - val_accuracy: 0.9660
Epoch 1/5
31/31 [==============================] - 12s 384ms/step - loss: 0.0361 - accuracy: 0.9897 - val_loss: 0.4074 - val_accuracy: 0.7282
Epoch 2/5
31/31 [==============================] - 9s 305ms/step - loss: 0.0231 - accuracy: 0.9910 - val_loss: 0.3046 - val_accuracy: 0.7799
Epoch 3/

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.10690359771251678, 0.244381844997406], [0.08362846821546555, 0.22898931801319122], [0.07095488160848618, 0.20549599826335907]]
Accuracy for iterations:  [[0.9252680540084839, 0.8563902974128723], [0.946609616279602, 0.8632020354270935], [0.9572292566299438, 0.8992589116096497]]
F1 for iterations:  [[0.9245748861778189, 0.8514425599764278], [0.9463300839036289, 0.8589810118026715], [0.9570748654616772, 0.8974500173269647]]
Precision for iterations:  [[0.9335533079035829, 0.8788159632177689], [0.9506441815206756, 0.8827556103007188], [0.9596707431635976, 0.909251160417791]]
Recall for iterations:  [[0.9252680475620089, 0.8563903218774976], [0.946609600046744, 0.8632020635035966], [0.9572292500511262, 0.8992588825110804]]


#### 5B 10%

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/2097221778.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/2097221778.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/2097221778.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/2097221778.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Bmed.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Bmed.hdf5')

Epoch 1/5
31/31 [==============================] - 11s 354ms/step - loss: 0.4953 - accuracy: 0.8018 - val_loss: 0.2969 - val_accuracy: 0.9195
Epoch 2/5
31/31 [==============================] - 9s 298ms/step - loss: 0.2683 - accuracy: 0.9282 - val_loss: 0.2432 - val_accuracy: 0.9398
Epoch 3/5
31/31 [==============================] - 11s 348ms/step - loss: 0.2414 - accuracy: 0.9402 - val_loss: 0.2296 - val_accuracy: 0.9420
Epoch 4/5
31/31 [==============================] - 11s 340ms/step - loss: 0.2305 - accuracy: 0.9422 - val_loss: 0.2228 - val_accuracy: 0.9432
Epoch 5/5
31/31 [==============================] - 11s 353ms/step - loss: 0.2243 - accuracy: 0.9425 - val_loss: 0.2189 - val_accuracy: 0.9433
Epoch 1/5
31/31 [==============================] - 11s 359ms/step - loss: 0.0371 - accuracy: 0.9904 - val_loss: 0.4993 - val_accuracy: 0.6725
Epoch 2/5
31/31 [==============================] - 9s 289ms/step - loss: 0.0230 - accuracy: 0.9911 - val_loss: 0.3812 - val_accuracy: 0.7009
Epoch 3/

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1181071475148201, 0.24809576570987701], [0.08594716340303421, 0.20506659150123596], [0.0689178928732872, 0.1927899867296219]]
Accuracy for iterations:  [[0.9239022731781006, 0.8438930511474609], [0.9465584754943848, 0.8827654123306274], [0.9548555612564087, 0.8966068625450134]]
F1 for iterations:  [[0.9231742327834022, 0.8376277834189282], [0.9462780117656485, 0.8800293158357178], [0.9546814595297561, 0.8946788398873966]]
Precision for iterations:  [[0.9325013096001444, 0.8705604423698192], [0.9506062984634849, 0.8967041757328372], [0.9575231853740371, 0.9070789639401701]]
Recall for iterations:  [[0.923902246632972, 0.8438930465741481], [0.9465584738087587, 0.8827653854537528], [0.95485553185895, 0.8966068444379859]]


#### 5B 25%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/3291834707.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/3291834707.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/3291834707.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/3291834707.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Bmed.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Bmed.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 156ms/step - loss: 0.5573 - accuracy: 0.7594 - val_loss: 0.4323 - val_accuracy: 0.8537
Epoch 2/5
31/31 [==============================] - 5s 152ms/step - loss: 0.4239 - accuracy: 0.8576 - val_loss: 0.4052 - val_accuracy: 0.8686
Epoch 3/5
31/31 [==============================] - 5s 150ms/step - loss: 0.4080 - accuracy: 0.8657 - val_loss: 0.3946 - val_accuracy: 0.8697
Epoch 4/5
31/31 [==============================] - 5s 162ms/step - loss: 0.3987 - accuracy: 0.8676 - val_loss: 0.3895 - val_accuracy: 0.8700
Epoch 5/5
31/31 [==============================] - 5s 158ms/step - loss: 0.3939 - accuracy: 0.8682 - val_loss: 0.3868 - val_accuracy: 0.8705
Epoch 1/5
31/31 [==============================] - 7s 219ms/step - loss: 0.0514 - accuracy: 0.9898 - val_loss: 0.5494 - val_accuracy: 0.6584
Epoch 2/5
31/31 [==============================] - 6s 188ms/step - loss: 0.0245 - accuracy: 0.9907 - val_loss: 0.4366 - val_accuracy: 0.6795
Epoch 3/5
31/

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1235242709517479, 0.24945220351219177], [0.11349524557590485, 0.23985375463962555], [0.09985047578811646, 0.24491889774799347]]
Accuracy for iterations:  [[0.9240118265151978, 0.8516675233840942], [0.92970871925354, 0.8634745478630066], [0.9374215006828308, 0.8655089735984802]]
F1 for iterations:  [[0.9232865704973329, 0.8463798930050868], [0.9291219478769307, 0.8591791071503254], [0.9369931311537052, 0.8613149899300455]]
Precision for iterations:  [[0.932586396290269, 0.8747490856128913], [0.9369985108197456, 0.8836588544370999], [0.9431179109978058, 0.8856218871314806]]
Recall for iterations:  [[0.9240118028572263, 0.851667514350069], [0.9297087265184493, 0.8634745331686405], [0.9374214847059511, 0.8655089733343021]]


#### 5C NODE 1 5%

In [32]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/591709827.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')


In [33]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [34]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C1med.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [35]:
global_model = build_model((24,1,1))

In [36]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [37]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [38]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1med.hdf5')

Epoch 1/5
16/16 [==============================] - 8s 482ms/step - loss: 0.6125 - accuracy: 0.7656 - val_loss: 0.4799 - val_accuracy: 0.7966
Epoch 2/5
16/16 [==============================] - 5s 320ms/step - loss: 0.3703 - accuracy: 0.8611 - val_loss: 0.2562 - val_accuracy: 0.9233
Epoch 3/5
16/16 [==============================] - 6s 358ms/step - loss: 0.2315 - accuracy: 0.9356 - val_loss: 0.2081 - val_accuracy: 0.9461
Epoch 4/5
16/16 [==============================] - 5s 310ms/step - loss: 0.2011 - accuracy: 0.9539 - val_loss: 0.1845 - val_accuracy: 0.9608
Epoch 5/5
16/16 [==============================] - 6s 392ms/step - loss: 0.1839 - accuracy: 0.9607 - val_loss: 0.1712 - val_accuracy: 0.9630
Epoch 1/5
14/14 [==============================] - 4s 302ms/step - loss: 0.0389 - accuracy: 0.9919 - val_loss: 0.6918 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 4s 309ms/step - loss: 0.0224 - accuracy: 0.9950 - val_loss: 0.7403 - val_accuracy: 0.7271
Epoch 3/5
14/

In [39]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07111847400665283, 0.13982725143432617], [0.056271787732839584, 0.10512643307447433], [0.04458804056048393, 0.07523000240325928]]
Accuracy for iterations:  [[0.9858964681625366, 0.9646152853965759], [0.9895483255386353, 0.9770035743713379], [0.9903955459594727, 0.9879568219184875]]
F1 for iterations:  [[0.9858995413258369, 0.9645747616430032], [0.9895534793036914, 0.9770561746141869], [0.9903999907930509, 0.9879729119059167]]
Precision for iterations:  [[0.9859353314564655, 0.9646811208681008], [0.9897732856068177, 0.9781476437653115], [0.990585726092373, 0.9882782308608551]]
Recall for iterations:  [[0.9858964620643315, 0.9646152728329579], [0.989548336206141, 0.9770035602702899], [0.9903955710070408, 0.9879568408050571]]


#### 5C NODE 1 10%

In [40]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/372464333.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')


In [41]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [42]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C1med.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [43]:
global_model = build_model((24,1,1))

In [44]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [45]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [46]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1med.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 233ms/step - loss: 0.6355 - accuracy: 0.7508 - val_loss: 0.5335 - val_accuracy: 0.7700
Epoch 2/5
16/16 [==============================] - 4s 221ms/step - loss: 0.4347 - accuracy: 0.8116 - val_loss: 0.3296 - val_accuracy: 0.9021
Epoch 3/5
16/16 [==============================] - 3s 215ms/step - loss: 0.2981 - accuracy: 0.9151 - val_loss: 0.2821 - val_accuracy: 0.9256
Epoch 4/5
16/16 [==============================] - 4s 222ms/step - loss: 0.2676 - accuracy: 0.9327 - val_loss: 0.2628 - val_accuracy: 0.9380
Epoch 5/5
16/16 [==============================] - 4s 238ms/step - loss: 0.2542 - accuracy: 0.9374 - val_loss: 0.2524 - val_accuracy: 0.9395
Epoch 1/5
14/14 [==============================] - 3s 237ms/step - loss: 0.0542 - accuracy: 0.9900 - val_loss: 0.5435 - val_accuracy: 0.7281
Epoch 2/5
14/14 [==============================] - 3s 240ms/step - loss: 0.0280 - accuracy: 0.9937 - val_loss: 0.8794 - val_accuracy: 0.7267
Epoch 3/5
14/

In [47]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06786628812551498, 0.13243860006332397], [0.05849339812994003, 0.10348297655582428], [0.05857144296169281, 0.11004547774791718]]
Accuracy for iterations:  [[0.9842677116394043, 0.967557966709137], [0.9895775318145752, 0.9798372387886047], [0.9896724820137024, 0.9768945574760437]]
F1 for iterations:  [[0.9842698489483893, 0.9675487599684763], [0.9895826687055785, 0.9798788785524012], [0.989677534573539, 0.9769476267394956]]
Precision for iterations:  [[0.9842837178181156, 0.9675495402370538], [0.989801253213745, 0.9807221705819592], [0.9898921713734905, 0.9780492537346367]]
Recall for iterations:  [[0.9842677261970844, 0.9675579452154327], [0.9895775511992755, 0.979837244786747], [0.9896724999269625, 0.9768945724042724]]


#### 5C NODE 1 25%

In [48]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/2868219575.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')


In [49]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [50]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C1med.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [51]:
global_model = build_model((24,1,1))

In [52]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [53]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [54]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1med.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 251ms/step - loss: 0.6367 - accuracy: 0.7120 - val_loss: 0.5530 - val_accuracy: 0.7273
Epoch 2/5
16/16 [==============================] - 4s 256ms/step - loss: 0.4974 - accuracy: 0.7913 - val_loss: 0.4441 - val_accuracy: 0.8451
Epoch 3/5
16/16 [==============================] - 4s 237ms/step - loss: 0.4405 - accuracy: 0.8482 - val_loss: 0.4195 - val_accuracy: 0.8645
Epoch 4/5
16/16 [==============================] - 4s 235ms/step - loss: 0.4258 - accuracy: 0.8618 - val_loss: 0.4114 - val_accuracy: 0.8682
Epoch 5/5
16/16 [==============================] - 4s 245ms/step - loss: 0.4159 - accuracy: 0.8639 - val_loss: 0.4039 - val_accuracy: 0.8682
Epoch 1/5
14/14 [==============================] - 4s 254ms/step - loss: 0.0808 - accuracy: 0.9898 - val_loss: 0.5626 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 4s 296ms/step - loss: 0.0269 - accuracy: 0.9937 - val_loss: 0.7068 - val_accuracy: 0.7270
Epoch 3/5
14/

In [55]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06564079970121384, 0.13114283978939056], [0.07139751315116882, 0.149677574634552], [0.0708380863070488, 0.11464150249958038]]
Accuracy for iterations:  [[0.9867802262306213, 0.9707004427909851], [0.9870139360427856, 0.9552604556083679], [0.9878100156784058, 0.9702826142311096]]
F1 for iterations:  [[0.9867837717367531, 0.9707073370850269], [0.9870214532090624, 0.9554150810797553], [0.9878167854153588, 0.9703644051403378]]
Precision for iterations:  [[0.9868390269264712, 0.9707216758611735], [0.9873599414442409, 0.9593962502272831], [0.9881152975602482, 0.9721658040422457]]
Recall for iterations:  [[0.9867802156066493, 0.9707004286856064], [0.9870139355517251, 0.955260480999782], [0.9878100441146397, 0.9702826418658723]]


#### 5C NODE 3 5%

In [56]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/927562370.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')


In [57]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [58]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C3med.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [59]:
global_model = build_model((24,1,1))

In [60]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [61]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [62]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3med.hdf5')

Epoch 1/5
16/16 [==============================] - 5s 294ms/step - loss: 0.5421 - accuracy: 0.7268 - val_loss: 1.7275 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 4s 256ms/step - loss: 0.2099 - accuracy: 0.9406 - val_loss: 3.4472 - val_accuracy: 0.1332
Epoch 3/5
16/16 [==============================] - 3s 193ms/step - loss: 0.0855 - accuracy: 0.9893 - val_loss: 3.9308 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 4s 221ms/step - loss: 0.0536 - accuracy: 0.9893 - val_loss: 2.0808 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 177ms/step - loss: 0.0367 - accuracy: 0.9900 - val_loss: 1.4119 - val_accuracy: 0.1722
Epoch 1/5
14/14 [==============================] - 3s 200ms/step - loss: 0.0225 - accuracy: 0.9949 - val_loss: 0.8312 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 194ms/step - loss: 0.0209 - accuracy: 0.9949 - val_loss: 0.5744 - val_accuracy: 0.7308
Epoch 3/5

In [63]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.10317321866750717, 0.10890663415193558], [0.13543936610221863, 0.15077562630176544], [0.18911060690879822, 0.1965211182832718]]
Accuracy for iterations:  [[0.9898039698600769, 0.9716994762420654], [0.9892196655273438, 0.93729567527771], [0.9655190110206604, 0.8878878355026245]]
F1 for iterations:  [[0.989808280414325, 0.9716685344645218], [0.9892236702120993, 0.9368286631446465], [0.9654729476641315, 0.885465503738214]]
Precision for iterations:  [[0.9899598662859304, 0.9717733720643744], [0.9893355259061654, 0.9401087061244824], [0.9660336299461918, 0.900551895405503]]
Recall for iterations:  [[0.9898039673960677, 0.9716994841241009], [0.9892196675333781, 0.937295647751217], [0.965519004353034, 0.8878878151565792]]


#### 5C NODE 3 10%

In [64]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_1864/4114672810.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')


In [65]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [66]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C3med.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [67]:
global_model = build_model((24,1,1))

In [68]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [69]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [70]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3med.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 197ms/step - loss: 0.5331 - accuracy: 0.7347 - val_loss: 1.7967 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 177ms/step - loss: 0.1934 - accuracy: 0.9616 - val_loss: 3.7692 - val_accuracy: 0.1330
Epoch 3/5
16/16 [==============================] - 3s 167ms/step - loss: 0.0809 - accuracy: 0.9893 - val_loss: 3.6043 - val_accuracy: 0.1340
Epoch 4/5
16/16 [==============================] - 3s 171ms/step - loss: 0.0508 - accuracy: 0.9893 - val_loss: 1.5942 - val_accuracy: 0.1454
Epoch 5/5
16/16 [==============================] - 3s 170ms/step - loss: 0.0366 - accuracy: 0.9890 - val_loss: 1.4648 - val_accuracy: 0.1721
Epoch 1/5
14/14 [==============================] - 3s 184ms/step - loss: 0.0222 - accuracy: 0.9949 - val_loss: 0.7898 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 195ms/step - loss: 0.0206 - accuracy: 0.9950 - val_loss: 0.5994 - val_accuracy: 0.7294
Epoch 3/5

In [71]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08947675675153732, 0.10681921988725662], [0.09570728987455368, 0.12717324495315552], [0.1559392809867859, 0.187766894698143]]
Accuracy for iterations:  [[0.9889713525772095, 0.977929949760437], [0.9893584251403809, 0.9637978672981262], [0.9884819984436035, 0.9178776144981384]]
F1 for iterations:  [[0.9889755858741193, 0.9779247832186535], [0.9893624729056961, 0.9637145823361895], [0.9884856073697068, 0.9168663752792593]]
Precision for iterations:  [[0.9890972505055337, 0.9779265600134767], [0.9894797778972626, 0.9641593904540664], [0.9885625507173158, 0.9238307470458704]]
Recall for iterations:  [[0.988971340091735, 0.9779299571314394], [0.9893584387507669, 0.9637978638378261], [0.9884819889567326, 0.9178776429557509]]


#### 5C NODE 3 25%

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16348/512859027.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')


In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C3med.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3med.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 198ms/step - loss: 0.5469 - accuracy: 0.7382 - val_loss: 1.6891 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 182ms/step - loss: 0.2158 - accuracy: 0.9386 - val_loss: 3.4275 - val_accuracy: 0.1330
Epoch 3/5
16/16 [==============================] - 2s 151ms/step - loss: 0.0869 - accuracy: 0.9893 - val_loss: 3.9847 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 186ms/step - loss: 0.0543 - accuracy: 0.9893 - val_loss: 2.0878 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 183ms/step - loss: 0.0371 - accuracy: 0.9900 - val_loss: 1.4474 - val_accuracy: 0.1666
Epoch 1/5
14/14 [==============================] - 3s 192ms/step - loss: 0.0220 - accuracy: 0.9950 - val_loss: 0.9446 - val_accuracy: 0.7268
Epoch 2/5
14/14 [==============================] - 3s 184ms/step - loss: 0.0205 - accuracy: 0.9950 - val_loss: 0.6951 - val_accuracy: 0.7280
Epoch 3/5

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09371992200613022, 0.11951834708452225], [0.2347034364938736, 0.2644708752632141], [0.11127892136573792, 0.1424078494310379]]
Accuracy for iterations:  [[0.9876420497894287, 0.9768763780593872], [0.9611732959747314, 0.8831650018692017], [0.9894022345542908, 0.9606553912162781]]
F1 for iterations:  [[0.9876460831178917, 0.9768677509101268], [0.9611111593452082, 0.8804515915801114], [0.9894061661916472, 0.9605432836813234]]
Precision for iterations:  [[0.9877296528638703, 0.9768764913923293], [0.9618557854207639, 0.8970246225318564], [0.989516031415439, 0.9612002217028675]]
Recall for iterations:  [[0.9876420579041164, 0.976876407759936], [0.9611732741242806, 0.8831650076291506], [0.9894022612404686, 0.9606553803676524]]
